In [27]:
# dependencies BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pymongo
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [28]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\jjel0\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


### NASA Mars News

In [29]:
# Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text.
    # Assign the text to variables that you can reference later.
url_mars_news = 'https://redplanetscience.com/'
browser.visit(url_mars_news)
html_mars_news = browser.html

In [4]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_news = bs(html_mars_news, 'html.parser')

In [30]:
list_text_list = soup_mars_news.find('div', class_='list_text')
content_title = list_text_list.find('div', class_='content_title').get_text()  
article_teaser_body = list_text_list.find('div', class_='article_teaser_body').get_text()

print(content_title)
article_teaser_body

# Collect the latest News Title and Paragraph Text.
# list_text_list = soup_mars_news.find_all('div', class_='list_text')

# content_title_strip_list = []
# article_teaser_body_strip_list = []

# # Loop through returned results (from 12-2-04)
# for result in list_text_list:
    
#     # Retrieve the title
#     content_title = result.find('div', class_='content_title')
#     content_title_strip = content_title.text.lstrip()
#     content_title_strip_list.append(content_title_strip)
    
#     # Retrieve the article body
#     article_teaser_body = result.find('div', class_='article_teaser_body')
#     article_teaser_body_strip = article_teaser_body.text.lstrip()
#     article_teaser_body_strip_list.append(article_teaser_body_strip)

# mars_news_dict = dict(zip(content_title_strip_list, article_teaser_body_strip_list))
# mars_news_dict

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light


'Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.'

### JPL Mars Space Images - Featured Image

In [6]:
# Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
url_mars_imag = 'https://spaceimages-mars.com'
browser.visit(url_mars_imag)
html_mars_imag = browser.html

In [7]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_imag = bs(html_mars_imag, 'html.parser')

In [8]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url 
    # string to a variable called `featured_image_url`.  Make sure to find the image url to the full size `.jpg` image.
    # reference C:\Users\jjel0\OneDrive\class_repo\gt-virt-atl-data-pt-06-2021-u-c-2\12-Web-Scraping-and-Document-Databases\2\Activities\07-Ins_Splinter
    # Example:
        # featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'
featured_image = soup_mars_imag.find('a', class_='showimg fancybox-thumbs')
# featured_image = soup_mars_imag.select_one('a.showimg fancybox-thumbs')

href = featured_image['href']
featured_image_url = 'https://spaceimages-mars.com/' + href
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [31]:
# Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing
    # facts about the planet including Diameter, Mass, etc.
url_mars_facts = 'https://galaxyfacts-mars.com'
browser.visit(url_mars_facts)
html_mars_facts = browser.html

In [10]:
# Create a Beautiful Soup object
soup_mars_facts = bs(html_mars_facts, 'lxml')

In [11]:
# Use Pandas to convert the data to a HTML table string.
html_mars_facts_html = pd.read_html(url_mars_facts)
html_mars_facts_html

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [33]:
html_mars_facts_html = pd.read_html(url_mars_facts)[1]

html_mars_facts_html.columns = ["description", "value"]
html_mars_facts_html.to_html(index=False)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

### Mars Hemispheres

In [24]:
# Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of 
    # Mars hemispheres.
url_mars_hemis = 'https://marshemispheres.com/'
browser.visit(url_mars_hemis)
html_mars_hemis = browser.html

In [22]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_hemis = bs(html_mars_hemis, 'html.parser')

# Print the image links
hemis_url_list = soup_mars_hemis.find_all('a', class_='itemLink product-item')
hemis_url_list

[<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="cerberus.html">
 <h3>Cerberus Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="schiaparelli.html">
 <h3>Schiaparelli Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="syrtis.html"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="syrtis.html">
 <h3>Syrtis Major Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="valles.html"><

In [25]:
hemisphere_image_urls = []
# Loop through returned results
for i in range(4):
    browser.find_by_css('a.product-item img')[i].click()
    hemisphere = {}
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    # Finally, we navigate backwards
    browser.back()
    
#     # Retrieve the title
#     content_title = result.find('h3')
#     content_title_hemis_list.append(content_title)
    
#     # Retrieve the images of the hemisphers
#     hemis_image = soup_mars_hemis.find('a', class_='itemLink product-item')
#     href = result['href']
#     hemis_image_url = 'https://marshemispheres.com/' + href
# #     print(hemis_image_url)
#     hemis_image_url_list.append(hemis_image_url)

# Not_none_values = filter(None.__ne__, content_title_hemis_list)

# content_title_hemis_listvalues = list(Not_none_values)


hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full
    # resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing
    # the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.


# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain
    # one dictionary for each hemisphere.
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg"},
#     {"title": "Cerberus Hemisphere", "img_url": "https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg"},
#     {"title": "Schiaparelli Hemisphere", "img_url": "https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg"},
#     {"title": "Syrtis Major Hemisphere", "img_url": "https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg"},
# ]


In [26]:
browser.quit()